In [ ]:
# Google Colab-only setup. No need to run this cell in other environments.

# Mount my Google Drive root folder
from google.colab import drive
drive.mount('/content/drive')

# cd to the location of this notebook
%cd 'drive/My Drive/Colab Notebooks/DropoutUncertaintyExps'
!ls

This file contains code to train dropout networks on the UCI datasets using the following algorithm:
1. Create 20 random splits of the training-test dataset.
2. For each split:
  1. Create a validation (val) set taking 20% of the training set.
  2. Get best hyperparameters: dropout_rate and tau by training on (train-val) set and testing on val set.
  3. Train a network on the entire training set with the best pair of hyperparameters.
  4. Get the performance (MC RMSE and log-likelihood) on the test set.
3. Report the averaged performance (Monte Carlo RMSE and log-likelihood) on all 20 splits.


In [1]:
import math
import numpy as np
import sys
import os
import datetime

# NN is defined in net/net.py
import net.net as net

Using Theano backend.


In [13]:
# Experiment settings
data_directory = "yacht"
epochs_multiplier = 10
num_hidden_layers = 1
results_file_directory_original = False

# We fix the random seed
np.random.seed(1)

In [14]:
_DATA_DIRECTORY_PATH = "./UCI_Datasets/" + data_directory + "/data/"
_DROPOUT_RATES_FILE = _DATA_DIRECTORY_PATH + "dropout_rates.txt"
_TAU_VALUES_FILE = _DATA_DIRECTORY_PATH + "tau_values.txt"
_DATA_FILE = _DATA_DIRECTORY_PATH + "data.txt"
_HIDDEN_UNITS_FILE = _DATA_DIRECTORY_PATH + "n_hidden.txt"
_EPOCHS_FILE = _DATA_DIRECTORY_PATH + "n_epochs.txt"
_INDEX_FEATURES_FILE = _DATA_DIRECTORY_PATH + "index_features.txt"
_INDEX_TARGET_FILE = _DATA_DIRECTORY_PATH + "index_target.txt"
_N_SPLITS_FILE = _DATA_DIRECTORY_PATH + "n_splits.txt"

In [15]:
# Results file directory setting
if results_file_directory_original:
    _RESULTS_FILE_ROOT = "./UCI_Datasets/" + data_directory # Original location in experiments.py
else:
    _RESULTS_FILE_ROOT = (
        "./test_results/" + data_directory + "/"
        + datetime.datetime.today().strftime('%Y%m%d%H%M') + "_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "/")
    
os.makedirs(_RESULTS_FILE_ROOT + "/results")

_RESULTS_VALIDATION_LL = _RESULTS_FILE_ROOT + "/results/validation_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_RMSE = _RESULTS_FILE_ROOT + "/results/validation_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_VALIDATION_MC_RMSE = _RESULTS_FILE_ROOT + "/results/validation_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

_RESULTS_TEST_LL = _RESULTS_FILE_ROOT + "/results/test_ll_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_TAU = _RESULTS_FILE_ROOT + "/results/test_tau_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_RMSE = _RESULTS_FILE_ROOT + "/results/test_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_MC_RMSE = _RESULTS_FILE_ROOT + "/results/test_MC_rmse_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"
_RESULTS_TEST_LOG = _RESULTS_FILE_ROOT + "/results/log_" + str(epochs_multiplier) + "_xepochs_" + str(num_hidden_layers) + "_hidden_layers.txt"

In [16]:
def _get_index_train_test_path(split_num, train = True):
    """
       Method to generate the path containing the training/test split for the given
       split number (generally from 1 to 20).
       @param split_num      Split number for which the data has to be generated
       @param train          Is true if the data is training data. Else false.
       @return path          Path of the file containing the requried data
    """
    if train:
        return _DATA_DIRECTORY_PATH + "index_train_" + str(split_num) + ".txt"
    else:
        return _DATA_DIRECTORY_PATH + "index_test_" + str(split_num) + ".txt" 

In [17]:
print ("Loading data and other hyperparameters...")
# We load the data

data = np.loadtxt(_DATA_FILE)

# We load the number of hidden units

n_hidden = np.loadtxt(_HIDDEN_UNITS_FILE).tolist()

# We load the number of training epocs

n_epochs = np.loadtxt(_EPOCHS_FILE).tolist()

# We load the indexes for the features and for the target

index_features = np.loadtxt(_INDEX_FEATURES_FILE)
index_target = np.loadtxt(_INDEX_TARGET_FILE)

X = data[ : , [int(i) for i in index_features.tolist()] ]
y = data[ : , int(index_target.tolist()) ]

# We iterate over the training test splits

n_splits = np.loadtxt(_N_SPLITS_FILE)
print ("Done.")

Loading data and other hyperparameters...
Done.


In [18]:
errors, MC_errors, lls, MC_preds, MC_vars = [], [], [], [], []
for split in range(int(n_splits)):

    # We load the indexes of the training and test sets
    print ('Loading file: ' + _get_index_train_test_path(split, train=True))
    print ('Loading file: ' + _get_index_train_test_path(split, train=False))
    index_train = np.loadtxt(_get_index_train_test_path(split, train=True))
    index_test = np.loadtxt(_get_index_train_test_path(split, train=False))

    X_train = X[ [int(i) for i in index_train.tolist()] ]
    y_train = y[ [int(i) for i in index_train.tolist()] ]
    
    X_test = X[ [int(i) for i in index_test.tolist()] ]
    y_test = y[ [int(i) for i in index_test.tolist()] ]

    X_train_original = X_train
    y_train_original = y_train
    num_training_examples = int(0.8 * X_train.shape[0])
    X_validation = X_train[num_training_examples:, :]
    y_validation = y_train[num_training_examples:]
    X_train = X_train[0:num_training_examples, :]
    y_train = y_train[0:num_training_examples]
    
    # Printing the size of the training, validation and test sets
    print ('Number of training examples: ' + str(X_train.shape[0]))
    print ('Number of validation examples: ' + str(X_validation.shape[0]))
    print ('Number of test examples: ' + str(X_test.shape[0]))
    print ('Number of train_original examples: ' + str(X_train_original.shape[0]))

    # List of hyperparameters which we will try out using grid-search
    dropout_rates = np.loadtxt(_DROPOUT_RATES_FILE).tolist()
    tau_values = np.loadtxt(_TAU_VALUES_FILE).tolist()

    # We perform grid-search to select the best hyperparameters based on the highest log-likelihood value
    best_network = None
    best_ll = -float('inf')
    best_tau = 0
    best_dropout = 0

    for dropout_rate in dropout_rates:
        for tau in tau_values:
            print ('Grid search step: Tau: ' + str(tau) + ' Dropout rate: ' + str(dropout_rate))
            network = net.net(X_train, y_train, ([ int(n_hidden) ] * num_hidden_layers),
                    normalize = True, n_epochs = int(n_epochs * epochs_multiplier), tau = tau,
                    dropout = dropout_rate)

            # We obtain the test RMSE and the test ll from the validation sets

            error, MC_error, ll = network.predict(X_validation, y_validation)
            if (ll > best_ll):
                best_ll = ll
                best_network = network
                best_tau = tau
                best_dropout = dropout_rate
                print ('Best log_likelihood changed to: ' + str(best_ll))
                print ('Best tau changed to: ' + str(best_tau))
                print ('Best dropout rate changed to: ' + str(best_dropout))
            
            # Storing validation results
            with open(_RESULTS_VALIDATION_RMSE, "a+") as myfile:
                myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
                myfile.write(repr(error) + '\n')

            with open(_RESULTS_VALIDATION_MC_RMSE, "a+") as myfile:
                myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
                myfile.write(repr(MC_error) + '\n')

            with open(_RESULTS_VALIDATION_LL, "a+") as myfile:
                myfile.write('Dropout_Rate: ' + repr(dropout_rate) + ' Tau: ' + repr(tau) + ' :: ')
                myfile.write(repr(ll) + '\n')

    # Storing test results
    best_network = net.net(X_train_original, y_train_original, ([ int(n_hidden) ] * num_hidden_layers),
                    normalize = True, n_epochs = int(n_epochs * epochs_multiplier), tau = best_tau,
                    dropout = best_dropout)
    error, MC_error, ll = best_network.predict(X_test, y_test)
    
    with open(_RESULTS_TEST_RMSE, "a+") as myfile:
        myfile.write(repr(error) + '\n')

    with open(_RESULTS_TEST_MC_RMSE, "a+") as myfile:
        myfile.write(repr(MC_error) + '\n')

    with open(_RESULTS_TEST_LL, "a+") as myfile:
        myfile.write(repr(ll) + '\n')

    with open(_RESULTS_TEST_TAU, "a+") as myfile:
        myfile.write(repr(best_network.tau) + '\n')

    print ("Tests on split " + str(split) + " complete.")
    errors += [error]
    MC_errors += [MC_error]
    lls += [ll]

Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_train_0.txt
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_test_0.txt
Number of training examples: 8592
Number of validation examples: 2149
Number of test examples: 1193
Number of train_original examples: 10741
Grid search step: Tau: 30000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 4us/step
Best log_likelihood changed to: 3.4073768083907217
Best tau changed to: 30000.0
Best dropout rate changed to: 0.005
Grid search step: Tau: 40000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.01
2149/2

2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.1
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.1
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.1
1193/1193 [==============================] - 0s 4us/step
Tests on split 4 complete.
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_train_5.txt
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_test_5.txt
Number of training examples: 8592
Number of validation examples: 2149
Number of test examples: 1193
Number of train_original examples: 10741
Grid search step: Tau: 30000.0 Dropout rate: 0.005
2149/2149 [==============================] 

2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 5us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.1
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.1
2149/2149 [===========================

1193/1193 [==============================] - 0s 5us/step
Tests on split 13 complete.
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_train_14.txt
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_test_14.txt
Number of training examples: 8592
Number of validation examples: 2149
Number of test examples: 1193
Number of train_original examples: 10741
Grid search step: Tau: 30000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 3us/step
Best log_likelihood changed to: 3.343777763666449
Best tau changed to: 30000.0
Best dropout rate changed to: 0.005
Grid search step: Tau: 40000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 7us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.005
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.01
2149/2149 [================

2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.01
2149/2149 [==============================] - 0s 7us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 3us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 7us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.05
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 30000.0 Dropout rate: 0.1
2149/2149 [==============================] - 0s 4us/step
Grid search step: Tau: 40000.0 Dropout rate: 0.1
2149/2149 [==============================] - 0s 7us/step
Grid search step: Tau: 50000.0 Dropout rate: 0.1
1193/1193 [==============================] - 0s 12us/step
Tests on split 18 complete.
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_train_19.txt
Loading file: ./UCI_Datasets/naval-propulsion-plant/data/index_test_19.txt
Number of trainin

In [12]:
with open(_RESULTS_TEST_LOG, "a+") as myfile:
    myfile.write('errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(errors), np.std(errors), np.std(errors)/math.sqrt(n_splits),
        np.percentile(errors, 50), np.percentile(errors, 25), np.percentile(errors, 75)))
    myfile.write('MC errors %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(MC_errors), np.std(MC_errors), np.std(MC_errors)/math.sqrt(n_splits),
        np.percentile(MC_errors, 50), np.percentile(MC_errors, 25), np.percentile(MC_errors, 75)))
    myfile.write('lls %f +- %f (stddev) +- %f (std error), median %f 25p %f 75p %f \n' % (
        np.mean(lls), np.std(lls), np.std(lls)/math.sqrt(n_splits), 
        np.percentile(lls, 50), np.percentile(lls, 25), np.percentile(lls, 75)))